In [1]:
import numpy as np
import time
import zhinst.toolkit as tk

In [2]:
mdc = tk.MultiDeviceController()
mdc.setup()

Successfully connected to data server at localhost8004 api version: 6


In [3]:
mdc.connect_hdawg("hd1", "dev8030", "1GbE")
mdc.connect_uhfqa("qa1", "dev2266", "1GbE")

Successfully connected to data server at localhost8004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE
Added HDAWG: hd1
Successfully connected to data server at localhost8004 api version: 6
Successfully connected to device DEV2266 on interface 1GBE
Added UHFQA: qa1


In [ ]:
mdc.hdawgs["hd1"].awgs[0].replace_waveform

In [7]:
mdc.uhfqas["qa1"].channels[0].result

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [4]:
trigger = mdc.hdawgs["hd1"].awgs[0]
awgs = mdc.hdawgs["hd1"].awgs[1:3]
readout = mdc.uhfqas["qa1"]

In [5]:
trigger.reset_queue()
trigger.output = "on"
trigger.set_sequence_params(
    sequence_type="Simple",
    trigger_mode="Send Trigger",
    alignment="Start with Trigger",
    period=20e-6,
    repetitions=1e7,
)
trigger.queue_waveform(np.ones(2400), np.ones(2400))
trigger.compile_and_upload_waveforms()

current length of queue: 1
Connection: change to dev8030, 0
Compilation successful
hdawg0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.012990236282348633 s


In [9]:
awgs[0].output = "on"
awgs[0].set_sequence_params(
    sequence_type="Rabi",
    trigger_mode="External Trigger",
    alignment="End with Trigger",
    period=20e-6,
    repetitions=1,
    pulse_width=5e-9,
)
awgs[0].compile()
awgs[0].enable_iq_modulation()
awgs[0].modulation_frequency = 91e6

Connection: change to dev8030, 0
Connection: change to dev8030, 1
Compilation successful
hdawg0: Sequencer status: ELF file uploaded


In [10]:
awgs[1].output = "on"
awgs[1].set_sequence_params(
    sequence_type="Rabi",
    trigger_mode="External Trigger",
    alignment="End with Trigger",
    period=20e-6,
    repetitions=1e7,
    pulse_width=5e-9,
)
awgs[1].enable_iq_modulation()
awgs[1].modulation_frequency = 105e6
awgs[1].compile()

Connection: change to dev8030, 2
Compilation successful
hdawg0: Sequencer status: ELF file uploaded


In [11]:
[ch.enable() for ch in readout.channels[:2]]
readout.channels[0].readout_frequency = 82e6
readout.channels[1].readout_frequency = 96e6

readout.awg.output = "on"
readout.awg.set_sequence_params(
    sequence_type="Readout",
    trigger_mode="External Trigger",
    alignment="Start with Trigger",
    period=20e-6,
    repetitions=1e7,
    trigger_delay=-100e-9
)
readout.awg.compile()

Connection: change to dev8030, 0
Connection: change to dev2266, 0
Compilation successful
uhfqa0: Sequencer status: ELF file uploaded


In [12]:
[awg.run() for awg in awgs]
readout.awg.run()
time.sleep(1)
trigger.run()

hdawg0: Started AWG 1!
hdawg0: Started AWG 2!
uhfqa0: Started AWG 0!
hdawg0: Started AWG 0!


In [13]:
[awg.stop() for awg in awgs]
readout.awg.stop()
trigger.stop()

hdawg0: Stopped AWG 1!
hdawg0: Stopped AWG 2!
uhfqa0: Stopped AWG 0!
hdawg0: Stopped AWG 0!
